# Import libraries and set up default plot params

#### Note, this cell picks the path from which you want to load tha data and to which you want to save all figures as your current working directory (`cwd`).
#### If you want to load from/save to a different path, edit the `path`.

In [ ]:
# Import libraries
import sys
import os

path = os.getcwd()

from collections import Counter

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

import random

import scipy.stats
from scipy.stats import binned_statistic_2d

# Import machine learning libraries
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, make_scorer
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.pipeline import Pipeline

# Set default tick label size
matplotlib.rcParams.update({'xtick.labelsize': 16})
matplotlib.rcParams.update({'ytick.labelsize': 16})

# Read in the data

In [ ]:
# Read in the csv file
df = pd.read_csv(path + '/' + 'haberman.data.csv')

#### Find the number of patients in each state, where state 1 means that the patient survived 5 years or longer, and state 2 tells us that the patient died within 5 years

Also, save the fraction of the dataset made up by the positive class to use later, where (for an imbalanced dataset) the smallest class represents the positive class

In [ ]:
# Count each state
states = df['STATE'].values
count = Counter(states)
min_v = len(df)+1
for k,v in count.items():
    p = v / len(states) * 100.0
    print('State={0:d}, Count={1:d}, Frac={2:.1f}%'.format(k, v, p))
    # Save the fraction of the dataset made up by the positive class
    if v < min_v:
        per_pos = p/100.0
    

#### Divide the columns into inputs (age, year, nodes) and outputs (state), and encode the output/target variable (y) to have values 0 and 1

In [ ]:
data = df.values

# split into input and output elements
X, y = data[:, :-1], data[:, -1]

# label encode the target variable to have the classes 0 and 1
y = LabelEncoder().fit_transform(y)


# Functions for evaluating the skill of the model

#### Use the Brier score, which calculates the mean squared error between the model predicted probabilities and the probabilities expected from the reference dataset
We calculate the reference dataset Brier score, where per_pos represents the expected baseline performance for the predictive model, and the model Brier score

We then calculate the model skill score, by comparing the Brier score for the reference and the model
By default, a skill score of 0.0 is a perfect score, but we invert such that 1.0 is a perfect score, and a score of 0.0 means the model performs exactly as well as the reference

In [ ]:
# Calculate Brier skill score (BSS)
# Use as a metric for evaluating the skill of the model based on the returned predicted probabilities
def brier_skill_score(y_true, y_prob):
    # Calculate Brier score for the reference (i.e., the dataset)
    ref_probs = [per_pos for _ in range(len(y_true))]
    bs_ref = brier_score_loss(y_true, ref_probs)
    # Calculate Brier score for the predictive model
    bs_model = brier_score_loss(y_true, y_prob)
    # Calculate skill score, by comparing the Brier score for the reference and the model
    print(1.0 - (bs_model / bs_ref))
    return 1.0 - (bs_model / bs_ref)


#### Use cross-validation, which uses a limited sample of a dataset in order to estimate the skill of a model, or how the model is expected to perform when used to make predictions on data not used in the training step

In [ ]:
# Evaluate the model
def evaluate_model(X, y, model):
    # k-fold cross-validation
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # Tell the scorer what metric to use
    metric = make_scorer(brier_skill_score, needs_proba=True)
    # Evaluate model
    scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
    return scores


#### As a check, we want to evaluate the baseline strategy of predicting the distribution of positive examples in the training dataset
We set the “strategy” to “prior” so the model will predict the prior probability of each class in the training dataset, which for the positive class we know is equal to per_pos


In [ ]:
# Define the reference model
model = DummyClassifier(strategy='prior')


#### We then evaluate the baseline model, which we expect to have a BSS of 0.0, i.e., the same as the reference model, because it IS the reference model

In [ ]:
# Evaluate the model
scores = evaluate_model(X, y, model)
# Summarize performance
print('Mean BSS: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))


# Evaluate test models

#### We will use the functions defined previously to evaluate a set of models that are known to be effective at predicting probabilities

In [ ]:
# Define models
models = [LogisticRegression(solver='lbfgs'), LinearDiscriminantAnalysis(),
          QuadraticDiscriminantAnalysis(), GaussianNB(), MultinomialNB(),
          GaussianProcessClassifier()]


#### We will compare the results from each model based on the mean Brier skill scoreand the distribution of the scores

In [ ]:
names, values = list(), list()
# Evaluate each model
for model in models:
    # Get a name for the model, used for plotting
    name = type(model).__name__[:10]
    # Evaluate the model and save results
    scores = evaluate_model(X, y, model)
    # Summarize performance
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    names.append(name)
    values.append(scores)
    

#### Create a box and whisker plot that shows the distribution of results from each model

In [ ]:
# Plot results
plt.boxplot(values, labels=names, showmeans=True)
plt.xticks(rotation=90)
plt.show()


# Evaluate test models with scaled inputs

#### It is good practice to scale data if the variables have different units of measure, as they do in this case

First, we use the StandardScaler, which is fit to the training dataset and applied within each k-fold cross-validation evaluation. Because scaling will shift the data to a mean of zero and unit standard deviation, we drop the MultinomialNB() model as it does not support negative input values.

In [ ]:
# Redefine models, drop the MultinomialNB() model
models = [LogisticRegression(solver='lbfgs'), LinearDiscriminantAnalysis(),
          QuadraticDiscriminantAnalysis(), GaussianNB(), GaussianProcessClassifier()]


#### Again, compare the results from each model

In [ ]:
names, values = list(), list()
# Evaluate each model
for model in models:
    # Get a name for the model, used for plotting
    name = type(model).__name__[:10]
    # Create a pipeline
    pip = Pipeline(steps=[('t', StandardScaler()),('m',model)])
    # Evaluate the model and save results
    scores = evaluate_model(X, y, pip)
    # Summarize performance
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    names.append(name)
    values.append(scores)
    

#### And create a box and whisker plot that shows the distribution of results from each model

The top three performing models show a similar spread in scores, suggesting that these models give the same general mapping of inputs to probabilities. We drop the QuadraticDiscriminantAnalysis() and GaussianNB() models.

In [ ]:
# Plot results
plt.boxplot(values, labels=names, showmeans=True)
plt.xticks(rotation=90)
plt.show()


#### Again, it is good practice to scale data if the variables have different units of measure, as they do in this case

Next, we use the PowerTransfor, which automatically determines how to make each variable more Gaussian. Because the power transform uses a log function, we also have to scale the dataset using a MinMaxScaler so that none of the values are negative.

In [ ]:
models = [LogisticRegression(solver='lbfgs'), LinearDiscriminantAnalysis(), GaussianProcessClassifier()]


In [ ]:
names, values = list(), list()
# Evaluate each model
for model in models:
    # Get a name for the model, used for plotting
    name = type(model).__name__[:10]
    # Create a pipeline
    pip = Pipeline(steps=[('t1', MinMaxScaler()), ('t2', PowerTransformer()),('m',model)])
    # Evaluate the model and save results
    scores = evaluate_model(X, y, pip)
    # Summarize performance
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    names.append(name)
    values.append(scores)
    

#### And create a box and whisker plot that shows the distribution of results from each model

The box and whisker plot suggests a smaller and more focused spread for LR compared to the LDA, which was the second-best performing method.

All methods still show skill on average, though the distribution of scores show runs that drop below 0.0 (no skill) in some cases.

#### We choose the LogisticRegression() model.

In [ ]:
# Plot results
plt.boxplot(values, labels=names, showmeans=True)
plt.xticks(rotation=90)
plt.show()


# Define the final model

In [ ]:
# Create a pipeline and fit the model
model = Pipeline(steps=[('t1', MinMaxScaler()), ('t2', PowerTransformer()),('m',LogisticRegression(solver='lbfgs'))])
model.fit(X, y)


# Run some test cases

#### Here, we'll check a few patients, pulled directly from the dataset, that we know survived (state 1) or did not survive (state 2) for longer than 5 years after their surgery. The model will output the probability of survival for each case.

In cases where we know that the patient selected did live for 5 years or longer after the surgery (state 1), the model predicts a probability of survival > 80%, where the predicted probability of survival for patients who did not live for 5 years after the surgery (state 2) was < 70%.

In [ ]:
# Survival (state 1) test cases
print('Known Survival Cases:')
data = [[31,59,2], [66,58,0], [34,60,1]]
for row in data:
    # Make model prediction
    yhat = model.predict_proba([row])
    # Probability of survival
    p_survive = yhat[0, 0] * 100
    # Summarize
    print('>data=%s, Survival=%.3f%%' % (row, p_survive))
    

In [ ]:
# Non-survival (state 2) test cases
print('Known Non-Survival Cases:')
data = [[44,64,6], [34,66,9], [38,69,21]]
for row in data:
    # Make model prediction
    yhat = model.predict_proba([row])
    ## Probability of survival
    p_survive = yhat[0, 0] * 100
    # Summarize
    print('data=%s, Survival=%.3f%%' % (row, p_survive))
    